In [3]:
import requests
from tqdm import tqdm
import pandas as pd
import numpy as np
import time
import my_utils as mu
import math
import random
import imp
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.graph_objs as go
import plotly.express as px
imp.reload(mu)
tqdm.pandas()

C:\Users\user\AppData\Local\Temp\ipykernel_9380\3501253536.py:9: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [2]:
raw_data = mu.get_rawdata('DIAMOND')

원시 데이터 생성중


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [03:14<00:00,  9.72s/it]


# 챔피언 시너지 또는 2:2 구도 분석

In [4]:
conn = mu.connect_mysql('icia')
for i in range(50):
    try:
        mu.mysql_execute('alter table lol_datas add primary key(gameId,participantId)', conn)
    except Exception as e:
        error=str(e)
        dupid=error.split("'")[1]
        dp = dupid.split('-')[0]
        mu.mysql_execute(f"delete from lol_datas where gameId ={repr(dp)}",conn)
conn.commit()
conn.close()

In [5]:
conn = mu.connect_mysql('icia')
df = pd.DataFrame(mu.mysql_execute_dict('select * from lol_datas', conn))
conn.close()

In [6]:
df['win']= df.apply(lambda x: 1 if x.win =='True' else 0, axis =1)

- blue팀 정글포지션 사람들만 꺼내서 [GAMEID,CHAMPIONNAME,WIN].RENAME(CHAMPIONID -> j_champ) \
   (테이블 이름 - blue_jungle)
- blue팀 미드포지션 사람들만 꺼내서 [GAMEID,CHAMPIONNAME,WIN].RENAME(CHAMPIONID -> m_champ)
    (테이블 이름 - blue_middle)
- 둘이 merge
    blue_team
- red팀도 동일한 포지션 동일하게 merge
    red_jungle
    red_top
    red_team

- 위에서 merge한 것을 각각 또 새롭게 테이블을 만들어서 enemy_j_champ, enemy_m_champ로 rename
    blue_tmp = blue_team.rename(enemy_j_champ,enemy_m_champ) 
    red_tmp

- red팀과 blue팀에 enemy_champ들을 merge해서 append
    tmp = merge()

- games와 win횟수를 구하고 win_rate 컬럼 추가한  result 테이블 만들기
    result =

In [7]:
df['teamPosition']

0             TOP
1          JUNGLE
2          MIDDLE
3          BOTTOM
4         UTILITY
           ...   
105235        TOP
105236     JUNGLE
105237     MIDDLE
105238     BOTTOM
105239    UTILITY
Name: teamPosition, Length: 105240, dtype: object

In [8]:
blue_jungle=df[(df['teamId']==100) & (df['teamPosition'] == 'JUNGLE')][['gameId','championName','win']].rename(columns = {'championName':'j_champ'})
blue_middle=df[(df['teamId']==100) & (df['teamPosition'] == 'MIDDLE')][['gameId','championName','win']].rename(columns = {'championName':'m_champ'})

In [9]:
red_jungle=df[(df['teamId']==200) & (df['teamPosition'] == 'JUNGLE')][['gameId','championName','win']].rename(columns = {'championName':'j_champ'})
red_middle=df[(df['teamId']==200) & (df['teamPosition'] == 'MIDDLE')][['gameId','championName','win']].rename(columns = {'championName':'m_champ'})

In [10]:
blue_team = pd.merge(blue_middle,blue_jungle,on=['gameId']).rename(columns = {'win_x':'j_win','win_y':'m_win'})
red_team = pd.merge(red_middle,red_jungle,on=['gameId']).rename(columns = {'win_x':'j_win','win_y':'m_win'})

In [11]:
blue_tmp = blue_team.rename(columns = {'j_champ':'enemy_j_champ','j_win':'enemy_j_win','m_champ':'enemy_m_champ','m_win':'enemy_m_win'})
red_tmp = red_team.rename(columns = {'j_champ':'enemy_j_champ','j_win':'enemy_j_win','m_champ':'enemy_m_champ','m_win':'enemy_m_win'})

In [12]:
blue_merge = pd.merge(blue_team,red_tmp,on=['gameId'])
red_merge = pd.merge(red_team,blue_tmp,on=['gameId'])

In [13]:
merge_df = blue_merge.append(red_merge).sort_values(by=['gameId']).reset_index(drop=True)

C:\Users\user\AppData\Local\Temp\ipykernel_9380\1252851882.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merge_df = blue_merge.append(red_merge).sort_values(by=['gameId']).reset_index(drop=True)


In [14]:
merge_df

,gameId,m_champ,j_win,j_champ,m_win,enemy_m_champ,enemy_j_win,enemy_j_champ,enemy_m_win
0,KR_4880568506,Irelia,0,Kayn,0,Galio,1,LeeSin,1
1,KR_4880568506,Galio,1,LeeSin,1,Irelia,0,Kayn,0
2,KR_4882960708,Sylas,1,Kayn,1,Ryze,0,Elise,0
3,KR_4882960708,Ryze,0,Elise,0,Sylas,1,Kayn,1
4,KR_4899886307,Lucian,0,Ekko,0,Camille,1,Graves,1
...,...,...,...,...,...,...,...,...,...
21041,KR_6412095217,Qiyana,1,Kindred,1,Anivia,0,Shaco,0
21042,KR_6413294766,Ahri,0,Zac,0,Cassiopeia,1,Sejuani,1
21043,KR_6413294766,Cassiopeia,1,Sejuani,1,Ahri,0,Zac,0
21044,KR_6413324482,Fizz,0,Kindred,0,Zed,1,Graves,1


In [15]:
merge1 = merge_df.groupby(['m_champ','j_champ','enemy_j_champ','enemy_m_champ'])[['gameId']].count().rename(columns={'gameId':'games'})

In [16]:
merge2 = merge_df.groupby(['m_champ','j_champ','enemy_j_champ','enemy_m_champ'])[['j_win']].sum().rename(columns={'j_win':'win'})

In [17]:
merge1.sort_values(by=['games'])

games
m_champ  j_champ enemy_j_champ enemy_m_champ       
Aatrox   Belveth Zed           Yone               1
Talon    Kindred Vi            MasterYi           1
                 Sylas         Ryze               1
                 Shyvana       Morgana            1
                 Shaco         Katarina           1
...                                             ...
Zed      Sylas   LeeSin        Leblanc            3
Katarina LeeSin  Ekko          Zed                3
Leblanc  LeeSin  Sylas         Zed                3
Yasuo    Gragas  LeeSin        Leblanc            3
Leblanc  LeeSin  Gragas        Yasuo              3

[20830 rows x 1 columns]

In [18]:
merge3 = pd.merge(merge1,merge2,on=['m_champ','j_champ','enemy_j_champ','enemy_m_champ'])

In [21]:
merge3['win_rate']=merge3['win']/merge3['games']
merge3.sort_values(by=['win'])[::-1][:30]

,,,,games,win,win_rate
m_champ,j_champ,enemy_j_champ,enemy_m_champ,,,
Akali,LeeSin,Nocturne,Zed,2,2,1.000000
Yone,Maokai,Udyr,Akali,2,2,1.000000
Katarina,Sylas,Belveth,Viktor,2,2,1.000000
Irelia,Sylas,LeeSin,Katarina,2,2,1.000000
Yasuo,LeeSin,Graves,Ahri,2,2,1.000000
Akali,MasterYi,Sylas,Yasuo,2,2,1.000000
Yone,Sylas,LeeSin,AurelionSol,2,2,1.000000
Gangplank,LeeSin,XinZhao,Malzahar,2,2,1.000000
Leblanc,LeeSin,Gragas,Yasuo,3,2,0.666667


In [93]:
blue_bot=df[(df['teamId']==100) & (df['teamPosition'] == 'BOTTOM')][['gameId','championName','win']].rename(columns = {'championName':'b_champ'})
blue_util=df[(df['teamId']==100) & (df['teamPosition'] == 'UTILITY')][['gameId','championName','win']].rename(columns = {'championName':'u_champ'})

In [94]:
red_bot=df[(df['teamId']==200) & (df['teamPosition'] == 'BOTTOM')][['gameId','championName','win']].rename(columns = {'championName':'b_champ'})
red_util=df[(df['teamId']==200) & (df['teamPosition'] == 'UTILITY')][['gameId','championName','win']].rename(columns = {'championName':'u_champ'})

In [95]:
blue_team = pd.merge(blue_bot,blue_util,on=['gameId']).rename(columns = {'win_x':'b_win','win_y':'u_win'})
red_team = pd.merge(red_bot,red_util,on=['gameId']).rename(columns = {'win_x':'b_win','win_y':'u_win'})

In [97]:
blue_tmp = blue_team.rename(columns = {'b_champ':'enemy_b_champ','b_win':'enemy_b_win','u_champ':'enemy_u_champ','u_win':'enemy_u_win'})
red_tmp = red_team.rename(columns = {'b_champ':'enemy_b_champ','b_win':'enemy_b_win','u_champ':'enemy_u_champ','u_win':'enemy_u_win'})

In [98]:
blue_merge = pd.merge(blue_team,red_tmp,on=['gameId'])
red_merge = pd.merge(red_team,blue_tmp,on=['gameId'])

In [99]:
merge_df = blue_merge.append(red_merge).sort_values(by=['gameId']).reset_index(drop=True)

C:\Users\user\AppData\Local\Temp\ipykernel_10896\1252851882.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merge_df = blue_merge.append(red_merge).sort_values(by=['gameId']).reset_index(drop=True)


In [102]:
merge1 = merge_df.groupby(['b_champ','u_champ','enemy_b_champ','enemy_u_champ'])[['gameId']].count().rename(columns={'gameId':'games'})
merge2 = merge_df.groupby(['b_champ','u_champ','enemy_b_champ','enemy_u_champ'])[['b_win']].sum().rename(columns={'b_win':'win'})
merge3 = pd.merge(merge1,merge2,on=['b_champ','u_champ','enemy_b_champ','enemy_u_champ'])

In [103]:
merge3['win_rate']=merge3['win']/merge3['games']
merge3.sort_values(by=['win'])[:30]

games  win  win_rate
b_champ  u_champ      enemy_b_champ enemy_u_champ                      
Kaisa    Nautilus     Caitlyn       Blitzcrank         1    0  0.000000
Jinx     Blitzcrank   Samira        Gragas             1    0  0.000000
Tristana Pyke         Ezreal        Shaco              1    0  0.000000
Jinx     Blitzcrank   Samira        Maokai             2    0  0.000000
                                    Nautilus           1    0  0.000000
...                                                  ...  ...       ...
Samira   Swain        Xayah         Soraka            16   16  1.000000
Kaisa    Blitzcrank   Ezreal        Leona             19   17  0.894737
         Sona         Ezreal        Yuumi             17   17  1.000000
Lucian   Nami         Ashe          MissFortune       17   17  1.000000
Ezreal   Heimerdinger Kaisa         Thresh            19   18  0.947368

[18776 rows x 3 columns]

In [108]:
df[(df['championName']=='Lucian')& (df['teamPosition']=='BOTTOM')][['win']]

,win
33,1
48,1
193,1
268,0
333,0
...,...
105283,0
105388,1
105418,1
105518,0


In [145]:
df.groupby(['gameId']).count()

,gameDuration,gameVersion,summonerName,summonerLevel,participantId,championName,champExperience,teamPosition,teamId,win,...,g_16,g_17,g_18,g_19,g_20,g_21,g_22,g_23,g_24,g_25
gameId,,,,,,,,,,,,,,,,,,,,,
KR_4880568506,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
KR_4882960708,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
KR_4899886307,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
KR_4906580554,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
KR_4924885897,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KR_6411067435,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
KR_6411088944,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
KR_6412095217,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10


In [146]:
df

,gameId,gameDuration,gameVersion,summonerName,summonerLevel,participantId,championName,champExperience,teamPosition,teamId,...,g_16,g_17,g_18,g_19,g_20,g_21,g_22,g_23,g_24,g_25
0,KR_6309025486,2136,13.1.489.3737,어둠의사신야스오,272,1,TahmKench,17323,TOP,100,...,4410,4862,5081,5699,5864,5986,6268,6790,6946,7259
1,KR_6309025486,2136,13.1.489.3737,리차드 Lim,417,2,Amumu,23196,JUNGLE,100,...,7915,8434,9224,10923,11045,11287,11666,12523,12651,13129
2,KR_6309025486,2136,13.1.489.3737,변종홍어,43,3,Leblanc,18307,MIDDLE,100,...,5578,5721,6100,6942,7196,7619,7880,8571,8716,9052
3,KR_6309025486,2136,13.1.489.3737,파주 최강 원딜러,251,4,Samira,17513,BOTTOM,100,...,7017,7736,8409,8807,9103,9470,10172,10739,11133,11909
4,KR_6309025486,2136,13.1.489.3737,God of moving,87,5,Ashe,17417,UTILITY,100,...,5986,6365,6672,7612,7753,8084,8680,9695,9884,10405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105815,KR_6381378912,1601,13.4.494.1549,MEROONA,52,6,Jax,10553,TOP,200,...,4007,4171,5598,5805,6135,6375,6641,6806,7780,7938
105816,KR_6381378912,1601,13.4.494.1549,라인주때염,299,7,Kindred,10194,JUNGLE,200,...,4650,5621,5874,6627,7249,7509,7971,8242,8561,8683
105817,KR_6381378912,1601,13.4.494.1549,지수는 공주,191,8,Galio,11597,MIDDLE,200,...,5239,5689,6018,6243,6445,6747,6899,7154,7495,7681
105818,KR_6381378912,1601,13.4.494.1549,정 신 혼 미 함,389,9,Sivir,9860,BOTTOM,200,...,4765,5082,5484,5681,5869,6252,6618,6879,7104,7463
